<h1>Run the below to initialise the functions.</h1>

In [1]:
import serial
import time
from twilio.rest import Client
account_sid = 'AC0b2a83c9d01a580c33a3c4bf70254d42'
auth_token = '96efd127b2de93f3a86f8fcb9ab146b8'
client = Client(account_sid, auth_token)

from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
%matplotlib inline
def W_init(shape,name=None):
    """Initialize weights as in paper"""
    values = rng.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)
#//TODO: figure out how to initialize layer biases in keras.
def b_init(shape,name=None):
    """Initialize bias as in paper"""
    values=rng.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)

input_shape = (105, 105, 1)
left_input = Input(input_shape)
right_input = Input(input_shape)
#build convnet to use in each siamese 'leg'
convnet = Sequential()
convnet.add(Conv2D(64,(10,10),activation='relu',input_shape=input_shape,
                   kernel_initializer=W_init,kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128,(7,7),activation='relu',
                   kernel_regularizer=l2(2e-4),kernel_initializer=W_init,bias_initializer=b_init))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(256,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(Flatten())
convnet.add(Dense(4096,activation="sigmoid",kernel_regularizer=l2(1e-3),kernel_initializer=W_init,bias_initializer=b_init))

#call the convnet Sequential model on each of the input tensors so params will be shared
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)
#layer to merge two encoded inputs with the l1 distance between them
L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#call this layer on list of two input tensors.
L1_distance = L1_layer([encoded_l, encoded_r])
prediction = Dense(1,activation='sigmoid',bias_initializer=b_init)(L1_distance)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

optimizer = Adam(0.00006)
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

siamese_net.count_params()
PATH = "/Users/prashanthsateesh/desktop/keras-oneshot-master" #CHANGE THIS - path where the pickled data is stored
import cv2,time
import numpy as np
from collections import deque
import argparse
import cmath
from PIL import Image
from matplotlib import pyplot as plt

print("Hello everyone.I am a bot powered by deep learning.I am capable of recognising gestures.So,here is how it works,you teach me 3 gestures,each of them one time to me and I will be able to classify them whenever you make such a gesture")
print("\n")

def calltoteachgesture1():
    teachgesture(1)
def calltoteachgesture2():
    teachgesture(2)
def calltoteachgesture3():
    teachgesture(3)




def teachgesture(gesturenumber):
    print("Teach me the "+str(gesturenumber)+"st "+ "gesture.")
    cap = cv2.VideoCapture(0)
    hand_cascade = cv2.CascadeClassifier('hand4.xml')
    array2=[]
    array1=[]

    c=0

    z = np.ones([720,1280,3], np.uint8)*255
    x = np.ones([720,1280,3], np.uint8)*255
    img = np.zeros((720,1280), np.uint8)
    img=img+255
    thickness = 15
    thickness2 = 45

    while(True):
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        hands = hand_cascade.detectMultiScale(gray, 1.1, 5)

        ##drawing of rectangular box around fist and filled circle at the centre of the box
        for(x,y,w,h) in hands:
            if (w*h>20000):
                cv2.rectangle(frame, (x,y),(x+w, y+h),(0, 255, 0), 2)
                l = int(w/2)
                m = int(h/2)
                cv2.circle(frame, (x+l,y+m), 10, (0, 0, 255), -1)
        
                if c==0 :
                    prev_x = x+l
                    prev_y = y+m
                    
                elif c>0 :
                    d = abs(cmath.sqrt((x+l-prev_x)*(x+l-prev_x)+(y+m-prev_y)*(y+m-prev_y)));
                      
                    if d<75 :
                        array1=(x+l,y+m)
                        prev_x = x+l
                        prev_y = y+m

                c = c+1;
        
        if array1!=[]:
            array2=array2+[array1]
                  
        for i in range(1, len(array2)):
            cv2.line(frame, (array2[i-1][0],array2[i-1][1]), (array2[i][0],array2[i][1]), (0, 0, 255), thickness)
            cv2.line(img, (array2[i-1][0],array2[i-1][1]), (array2[i][0],array2[i][1]), (1, 1, 1), thickness2)

        frame = cv2.flip( frame, 1 )
        cv2.imshow('Frame',frame)
        key=cv2.waitKey(1)
        if key==ord('q'):
            cap.release()
            break
    print("If you want to save this gesture press Y else press N to reteach")
    option=str(input("Enter your choice: "))
    if option=='y':
        fnumber= open(str(gesturenumber)+"/"+"filenumber.txt","r")


        #img = np.zeros((720,1280), np.uint8)
        #img=img+255
        pts = np.array(array2, np.int32)
        pts = pts.reshape((-1,1,2))
        #cv2.polylines(img,[pts],True,(0,0,0))
        #cv2.line(img, (pts), (0, 0, 0), thickness)

        img = cv2.flip( img, 1 )

        if fnumber.readline()=='':
            fnumber.close()
            fnumber= open(str(gesturenumber)+"/"+"filenumber.txt","w")
            cv2.imwrite(str(gesturenumber)+"/"+str(1)+".png",img)
            foo = Image.open(str(gesturenumber)+"/"+str(1)+".png")
            foo = foo.resize((105,105),Image.ANTIALIAS)
            foo.save(str(gesturenumber)+"/"+str(1)+".png",quality=100)
            fnumber.write("1\n")
            fnumber.close()
        else:
            fnumber= open(str(gesturenumber)+"/"+"filenumber.txt","r")

            temp=fnumber.readline()
            fnumber.close()
            temp=int(temp)
            temp=temp+1
            fnumber= open(str(gesturenumber)+"/"+"filenumber.txt","w")
            cv2.imwrite(str(gesturenumber)+"/"+str(temp)+".png",img)
            foo = Image.open(str(gesturenumber)+"/"+str(temp)+".png")
            foo = foo.resize((105,105),Image.ANTIALIAS)
            foo.save(str(gesturenumber)+"/"+str(temp)+".png",quality=100)
            fnumber.write(str(temp)+"\n")
            fnumber.close()
    elif option=='n':
        calltoteachgesture1()
def gesture():
    print("Make the geture that you want to classify")
    cap = cv2.VideoCapture(0)
    hand_cascade = cv2.CascadeClassifier('hand4.xml')
    array2=[]
    array1=[]

    c=0

    z = np.ones([720,1280,3], np.uint8)*255
    x = np.ones([720,1280,3], np.uint8)*255
    img = np.zeros((720,1280), np.uint8)
    img=img+255
    thickness = 15
    thickness2 = 45

    while(True):
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        hands = hand_cascade.detectMultiScale(gray, 1.1, 5)

        ##drawing of rectangular box around fist and filled circle at the centre of the box
        for(x,y,w,h) in hands:
            if (w*h>20000):
                cv2.rectangle(frame, (x,y),(x+w, y+h),(0, 255, 0), 2)
                l = int(w/2)
                m = int(h/2)
                cv2.circle(frame, (x+l,y+m), 10, (0, 0, 255), -1)
        
                if c==0 :
                    prev_x = x+l
                    prev_y = y+m
                    
                elif c>0 :
                    d = abs(cmath.sqrt((x+l-prev_x)*(x+l-prev_x)+(y+m-prev_y)*(y+m-prev_y)));
                      
                    if d<75 :
                        array1=(x+l,y+m)
                        prev_x = x+l
                        prev_y = y+m

                c = c+1;
        
        if array1!=[]:
            array2=array2+[array1]
                  
        for i in range(1, len(array2)):
            cv2.line(frame, (array2[i-1][0],array2[i-1][1]), (array2[i][0],array2[i][1]), (0, 0, 255), thickness)
            cv2.line(img, (array2[i-1][0],array2[i-1][1]), (array2[i][0],array2[i][1]), (1, 1, 1), thickness2)

        frame = cv2.flip( frame, 1 )
        cv2.imshow('Frame',frame)
        key=cv2.waitKey(1)
        if key==ord('q'):
            cap.release()
            break
    
    fnumber= open(str(4)+"/"+"filenumber.txt","r")


        #img = np.zeros((720,1280), np.uint8)
        #img=img+255
    pts = np.array(array2, np.int32)
    pts = pts.reshape((-1,1,2))
        #cv2.polylines(img,[pts],True,(0,0,0))
        #cv2.line(img, (pts), (0, 0, 0), thickness)

    img = cv2.flip( img, 1 )

    if fnumber.readline()=='':
            fnumber.close()
            fnumber= open(str(4)+"/"+"filenumber.txt","w")
            cv2.imwrite(str(4)+"/"+str(1)+".png",img)
            foo = Image.open(str(4)+"/"+str(1)+".png")
            foo = foo.resize((105,105),Image.ANTIALIAS)
            foo.save(str(4)+"/"+str(1)+".png",quality=100)
            fnumber.write("1\n")
            fnumber.close()
    else:
            fnumber= open(str(4)+"/"+"filenumber.txt","r")

            temp=fnumber.readline()
            fnumber.close()
            temp=int(temp)
            temp=temp+1
            fnumber= open(str(4)+"/"+"filenumber.txt","w")
            cv2.imwrite(str(4)+"/"+str(temp)+".png",img)
            foo = Image.open(str(4)+"/"+str(temp)+".png")
            foo = foo.resize((105,105),Image.ANTIALIAS)
            foo.save(str(4)+"/"+str(temp)+".png",quality=100)
            fnumber.write(str(temp)+"\n")
            fnumber.close()
        

Using TensorFlow backend.


Hello everyone.I am a bot powered by deep learning.I am capable of recognising gestures.So,here is how it works,you teach me 3 gestures,each of them one time to me and I will be able to classify them whenever you make such a gesture




<h1>Call below to teach gestures.</h1>

In [7]:
calltoteachgesture1()
calltoteachgesture2()
calltoteachgesture3()
print("All gestures have been learnt.\n")

Teach me the 1st gesture.
If you want to save this gesture press Y else press N to reteach
Enter your choice: y
Teach me the 2st gesture.
If you want to save this gesture press Y else press N to reteach
Enter your choice: y
Teach me the 3st gesture.
If you want to save this gesture press Y else press N to reteach
Enter your choice: y
All gestures have been learnt.



<h1>Call below to execute one of the applications.</h1>

In [10]:
print("Make a gesture to exectue any one of the applications.\n")
gesture()
img = Image.open( '/Users/prashanthsateesh/desktop/keras-oneshot-master/4/'+'1' + '.png' )
data = np.array( img, dtype='uint8' )
input1=np.asarray(data).reshape(1,105,105,1)
siamese_net.load_weights( "/Users/prashanthsateesh/desktop/keras-oneshot-master/weights_4")
sumgesture1=0
final=[]
for letter in os.listdir("/Users/prashanthsateesh/desktop/keras-oneshot-master/1"):
    if letter.find(".DS_Store")==-1 & letter.find("filenumber")==-1:
        img2 = Image.open("/Users/prashanthsateesh/desktop/keras-oneshot-master/1/"+str(letter))
        data2 = np.array(img2, dtype='uint8' )
        input2=np.asarray(data2).reshape(1,105,105,1)
        inputs=[input1,input2]
        final=final+[siamese_net.predict(inputs)[0][0]]
for letter in os.listdir("/Users/prashanthsateesh/desktop/keras-oneshot-master/2"):
    if letter.find(".DS_Store")==-1 & letter.find("filenumber")==-1:
        img2 = Image.open("/Users/prashanthsateesh/desktop/keras-oneshot-master/2/"+str(letter))
        data2 = np.array(img2, dtype='uint8' )
        input2=np.asarray(data2).reshape(1,105,105,1)
        inputs=[input1,input2]
        final=final+[siamese_net.predict(inputs)[0][0]]
for letter in os.listdir("/Users/prashanthsateesh/desktop/keras-oneshot-master/3"):
    if letter.find(".DS_Store")==-1 & letter.find("filenumber")==-1:
        img2 = Image.open("/Users/prashanthsateesh/desktop/keras-oneshot-master/3/"+str(letter))
        data2 = np.array(img2, dtype='uint8' )
        input2=np.asarray(data2).reshape(1,105,105,1)
        inputs=[input1,input2]
        final=final+[siamese_net.predict(inputs)[0][0]]
print("The application to execute is: "+str(final.index(max(final))+1))
if(final.index(max(final))+1==1):
   
    call = client.calls.create(
                        url='http://demo.twilio.com/docs/voice.xml',
                        to='+918582908625',
                        from_='+12483270277'
                    )

    print(call.sid)
   
       
elif(final.index(max(final))+1==2):
   
    message = client.messages \
                .create(
                     body="Call Dr.Amartya immediately..",
                     from_='+12483270277',
                     to='+918582908625'
                 )
elif(final.index(max(final))+1==3):
 
             arduinoData=serial.Serial('/dev/cu.usbmodem1411',9600)

             def led_on():
                arduinoData.write(b'1')
             time.sleep(2)
             led_on()


Make a gesture to exectue any one of the applications.

Make the geture that you want to classify
The application to execute is: 3
